In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
#from tensorflow.keras import datasets, layers, models

In [2]:
#加载数据集
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)
print(mnist.train.images.shape,mnist.train.labels.shape)
print(mnist.test.images.shape,mnist.test.labels.shape)
print(mnist.validation.images.shape,mnist.validation.labels.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10)
(10000, 784) (10000, 10)
(5000, 784) (5000, 10)


In [3]:
#单层线性模型+softmax+交叉熵损失函数+梯度下降优化
x=tf.placeholder(tf.float32,shape=[None,784])
W=tf.Variable(tf.zeros([784,10]))
b=tf.Variable(tf.zeros([10]))
y=tf.nn.softmax(tf.matmul(x,W)+b)
y_ = tf.placeholder(tf.float32,[None,10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(1000):
        batch_xs,batch_ys = mnist.train.next_batch(100)
        train_step.run({x:batch_xs,y_:batch_ys})
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    print(accuracy.eval({x:mnist.test.images,y_:mnist.test.labels}))

0.9158


In [4]:
#感知机+softmax+交叉熵损失函数+adam优化
x=tf.placeholder(tf.float32,shape=[None,784])
keep_prob = tf.placeholder(tf.float32)

W1=tf.Variable(tf.truncated_normal([784,300],stddev=0.1))
b1=tf.Variable(tf.zeros([300]))
W2=tf.Variable(tf.truncated_normal([300,10],stddev=0.1))
b2=tf.Variable(tf.zeros([10]))

hidden1 = tf.nn.dropout(tf.nn.relu(tf.matmul(x,W1)+b1),keep_prob)
#hidden1 = tf.nn.tanh(tf.matmul(x,W1)+b1)
y=tf.nn.softmax(tf.matmul(hidden1,W2)+b2)
y_ = tf.placeholder(tf.float32,[None,10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(3000):
        batch_xs,batch_ys = mnist.train.next_batch(100)
        train_step.run({x:batch_xs,y_:batch_ys,keep_prob:0.75})
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    print(accuracy.eval({x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))

0.98


In [21]:
#cnn tf.nn实现
#tf.device('/gpu:0')
x=tf.placeholder(tf.float32,shape=[None,784])
y_ = tf.placeholder(tf.float32,[None,10])
keep_prob= tf.placeholder(tf.float32)
x_image=tf.reshape(x,[-1,28,28,1])

W_conv1=tf.Variable(tf.truncated_normal([5,5,1,32],stddev=0.1))
b_conv1=tf.Variable(tf.constant(0.1,shape=[32]))
h_conv1=tf.nn.relu(tf.nn.conv2d(x_image,W_conv1,strides=[1,1,1,1],padding='SAME')+b_conv1)
h_pool1=tf.nn.max_pool(h_conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W_conv2=tf.Variable(tf.truncated_normal([5,5,32,64],stddev=0.1))
b_conv2=tf.Variable(tf.constant(0.1,shape=[64]))
h_conv2=tf.nn.relu(tf.nn.conv2d(h_pool1,W_conv2,strides=[1,1,1,1],padding='SAME')+b_conv2)
h_pool2=tf.nn.max_pool(h_conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W_fc1=tf.Variable(tf.truncated_normal([7*7*64,1024],stddev=0.1))
b_fc1=tf.Variable(tf.constant(0.1,shape=[1024]))
h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

W_fc2=tf.Variable(tf.truncated_normal([1024,10],stddev=0.1))
b_fc2=tf.Variable(tf.constant(0.1,shape=[10]))
# y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv),reduction_indices=[1]))
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
 
    # 定义损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(20000):
        batch_xs,batch_ys = mnist.train.next_batch(50)
        if i%100==0:
            train_accuracy=accuracy.eval(feed_dict={x:batch_xs,y_:batch_ys,keep_prob:1.0})
            print('step %d,training accuracy %g'%(i,train_accuracy))
        
        train_step.run({x:batch_xs,y_:batch_ys,keep_prob:0.5})
    print(accuracy.eval({x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))

step 0,training accuracy 0.08
step 100,training accuracy 0.84
step 200,training accuracy 0.84
step 300,training accuracy 0.84
step 400,training accuracy 0.96
step 500,training accuracy 1
step 600,training accuracy 0.98
step 700,training accuracy 0.96
step 800,training accuracy 0.94
step 900,training accuracy 0.94
step 1000,training accuracy 0.9
step 1100,training accuracy 0.98
step 1200,training accuracy 0.98
step 1300,training accuracy 0.96
step 1400,training accuracy 0.96
step 1500,training accuracy 0.98
step 1600,training accuracy 0.96
step 1700,training accuracy 0.98
step 1800,training accuracy 0.96
step 1900,training accuracy 0.98
step 2000,training accuracy 0.96
step 2100,training accuracy 0.96
step 2200,training accuracy 0.96
step 2300,training accuracy 0.98
step 2400,training accuracy 1
step 2500,training accuracy 1
step 2600,training accuracy 0.98
step 2700,training accuracy 1
step 2800,training accuracy 1
step 2900,training accuracy 0.96
step 3000,training accuracy 0.98
step 

In [10]:
#cnn函数封装
# 定义输入和输出的占位符
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
 
# 定义通用函数
def weight_variable(shape):
    # 截断正态分布 标准方差为0.1
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
 
def bias_variable(shape):
    # 设为非零避免死神经元
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
 
def conv2d(x, W):
    # 卷积不改变输入的shape
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
 
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                              strides=[1, 2, 2, 1], padding='SAME')
 
sess = tf.InteractiveSession()
 
# 构建模型
 
# 把输入变换成一个4d的张量，第二三个对应的是图片的长和宽，第四个参数对应的颜色
x_image = tf.reshape(x, [-1, 28, 28, 1])
 
# 计算32个特征，每5*5patch,第一二个参数指的是patch的size，第三个参数是输入的channels，第四个参数是输出的channels
W_conv1 = weight_variable([5, 5, 1, 32])
# 偏差的shape应该和输出的shape一致，所以也是32
b_conv1 = bias_variable([32])
 
# 28*28的图片卷积时步长为1，随意卷积后大小不变，按2*2最大值池化，相当于从2*2块中提取一个最大值，
# 所以池化后大小为[28/2,28/2] = [14,14]，第二次池化后为[14/2,14/2] = [7,7]
 
# 对数据做卷积操作
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# max_pool_2x2之后，图片变成14*14
h_pool1 = max_pool_2x2(h_conv1)
 
    # 在以前的基础上，生成了64个特征
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
 
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    # max_pool_2x2之后，图片变成7*7
h_pool2 = max_pool_2x2(h_conv2)
 
    # 构造一个全连接的神经网络，1024个神经元
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
    # 输出为1024
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
 
    # 做Dropout操作
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
 
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
 
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
 
    # 定义损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    # 定义优化函数
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    # 计算准确率
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    # 初始化参数
sess.run(tf.global_variables_initializer())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
print(accuracy.eval({x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))

step 0, training accuracy 0.08
step 100, training accuracy 0.78
step 200, training accuracy 0.94
step 300, training accuracy 0.84
step 400, training accuracy 0.92
step 500, training accuracy 0.88
step 600, training accuracy 0.98
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.98
step 1000, training accuracy 0.9
step 1100, training accuracy 0.94
step 1200, training accuracy 0.92
step 1300, training accuracy 0.94
step 1400, training accuracy 0.94
step 1500, training accuracy 0.98
step 1600, training accuracy 1
step 1700, training accuracy 0.96
step 1800, training accuracy 1
step 1900, training accuracy 0.98
step 2000, training accuracy 0.96
step 2100, training accuracy 0.96
step 2200, training accuracy 0.96
step 2300, training accuracy 1
step 2400, training accuracy 0.94
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 0.98
step 2800, training accuracy 1
step 2900, training accuracy 0.98
step 300

In [ ]:
#cnn tf.keras高阶封装API编写
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1),padding='SAME'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu',padding='SAME'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

tf.saved_model.save(model, "/model/")